<h1>Utilizando tecnicas para redução da dimensionalidade</h1>

In [2]:
# Import the SHAP library - https://github.com/slundberg/shap
import tensorflow as tf
import sklearn
import pandas as pd
import numpy as np
from numpy import set_printoptions
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel

from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import ExtraTreesClassifier

2024-08-25 23:45:06.820789: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-25 23:45:06.822037: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-25 23:45:06.856809: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-25 23:45:07.060314: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-25 23:45:08.231538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

### Leitura dos dados

In [3]:
#Importar o arquivo
df_ef1 = pd.read_csv('datasets/DADOS_INEP_EF_AI.csv', encoding="latin-1", sep = ';')
df_ef2 = pd.read_csv('datasets/DADOS_INEP_EF_AF.csv', encoding="latin-1", sep = ';')

#Definir a meta do IDEB para a etapa do ensino
meta_ideb_ef1 = 6.0
meta_ideb_ef2 = 5.5



### 

Pré-processamento

In [4]:
## ENSINO FUNDAMENTAL - ANOS INICIAIS ##############################################################
#Exclui colunas cadastrais
df_ef1.drop(['ID_ESCOLA','NU_ANO_CENSO','SG_UF','NO_MUNICIPIO','CO_MUNICIPIO','NO_ENTIDADE','DT_ANO_LETIVO_INICIO','DT_ANO_LETIVO_TERMINO'], axis=1, inplace=True)

#Transformando em zeros colunas com '--'
df_ef1 = df_ef1.replace('--', '0', regex=True)

#Transformação necessária, alterar o valor não informado (88888.000000) por zero
df_ef1 = df_ef1.replace(88888, 0, regex=True)

#mudando o separador decimal para '.' e convertendo o tipo para float
df_ef1 = df_ef1.replace(',', '.', regex=True).astype(float)

#Resetando o índice
df_ef1.reset_index(drop=True, inplace=True)


## ENSINO FUNDAMENTAL - ANOS FINAIS ##############################################################
#Exclui colunas cadastrais
df_ef2.drop(['ID_ESCOLA','NU_ANO_CENSO','SG_UF','NO_MUNICIPIO','CO_MUNICIPIO','NO_ENTIDADE','DT_ANO_LETIVO_INICIO','DT_ANO_LETIVO_TERMINO'], axis=1, inplace=True)

#Transformando em zeros colunas com '--'
df_ef2 = df_ef2.replace('--', '0', regex=True)

#Transformação necessária, alterar o valor não informado (88888.000000) por zero
df_ef2 = df_ef2.replace(88888, 0, regex=True)

#mudando o separador decimal para '.' e convertendo o tipo para float
df_ef2 = df_ef2.replace(',', '.', regex=True).astype(float)

#Resetando o índice
df_ef2.reset_index(drop=True, inplace=True)

In [5]:
#Transformar o IDEB na coluna com a classe 1 (atingiu o IDEB de 2021) ou 0 (não atingiu o ideb de 2021)
df_ef1["VL_OBSERVADO_2021"] = np.where(df_ef1['VL_OBSERVADO_2021']>= meta_ideb_ef1, 1, 0)
print(df_ef1['VL_OBSERVADO_2021'].value_counts())

#Transformar o IDEB na coluna com a classe 1 (atingiu o IDEB de 2021) ou 0 (não atingiu o ideb de 2021)
df_ef2["VL_OBSERVADO_2021"] = np.where(df_ef2['VL_OBSERVADO_2021']>= meta_ideb_ef2, 1, 0)
print(df_ef2['VL_OBSERVADO_2021'].value_counts())

0    21385
1    10899
Name: VL_OBSERVADO_2021, dtype: int64
0    16784
1     5208
Name: VL_OBSERVADO_2021, dtype: int64


In [6]:
#Junta os dataframes em um só
df = pd.concat([df_ef1, df_ef2], axis=0)

#Exclusão de colunas
df.drop(['IN_INF', 'IN_INF_CRE', 'IN_INF_PRE', 'IN_BAS', 'IN_FUND_AI', 'IN_FUND_AF', 'IN_MED', 'QT_MAT_INF', 'QT_MAT_INF_CRE', 'QT_MAT_INF_PRE', 'QT_MAT_BAS', 'QT_MAT_FUND_AI', 'QT_MAT_FUND_AF', 'QT_MAT_MED', 'QT_MAT_PROF', 'QT_MAT_PROF_TEC', 'QT_MAT_EJA', 'QT_MAT_EJA_FUND', 'QT_MAT_EJA_MED', 'QT_MAT_ESP', 'QT_MAT_ESP_CC', 'QT_MAT_ESP_CE', 'QT_MAT_BAS_0_3', 'QT_MAT_BAS_4_5', 'QT_MAT_BAS_6_10', 'QT_MAT_BAS_11_14', 'QT_MAT_BAS_15_17', 'QT_MAT_BAS_18_MAIS', 'QT_DOC_BAS', 'QT_DOC_INF_CRE', 'QT_DOC_INF_PRE', 'QT_DOC_FUND_AI', 'QT_DOC_FUND_AF', 'QT_DOC_MED', 'QT_TUR_BAS', 'QT_TUR_INF_CRE', 'QT_TUR_INF_PRE', 'QT_TUR_FUND', 'QT_TUR_FUND_AI', 'QT_TUR_FUND_AF', 'QT_TUR_MED', 'QT_MAT_INF_INT','QT_MAT_INF_CRE_INT','QT_MAT_INF_PRE_INT','QT_MAT_FUND_AI_INT','QT_MAT_FUND_AF_INT','QT_MAT_MED_INT'], axis=1, inplace=True)

#Exclusão de mais colunas
df.drop(['QT_MAT_BAS_D','QT_MAT_BAS_N','QT_MAT_BAS_EAD','QT_MAT_BAS_ND','QT_MAT_BAS_BRANCA','QT_MAT_BAS_PRETA','QT_MAT_BAS_PARDA','QT_MAT_BAS_AMARELA','QT_MAT_BAS_INDIGENA','QT_MAT_BAS_FEM','QT_MAT_BAS_MASC'], axis=1, inplace=True)

#Exclusão de colunas constantes
df.drop(['TP_CATEGORIA_ESCOLA_PRIVADA','TP_SITUACAO_FUNCIONAMENTO','IN_CONVENIADA_PP','TP_CONVENIO_PODER_PUBLICO','IN_MANT_ESCOLA_PRIVADA_EMP','IN_MANT_ESCOLA_PRIVADA_ONG','IN_MANT_ESCOLA_PRIVADA_OSCIP','IN_MANT_ESCOLA_PRIV_ONG_OSCIP','IN_MANT_ESCOLA_PRIVADA_SIND','IN_MANT_ESCOLA_PRIVADA_SIST_S','IN_MANT_ESCOLA_PRIVADA_S_FINS','IN_LOCAL_FUNC_SALAS_EMPRESA','IN_LOCAL_FUNC_TEMPLO_IGREJA','IN_LOCAL_FUNC_CASA_PROFESSOR'], axis=1, inplace=True)

df.reset_index(drop=True, inplace=True)



df


,TX_APR,TX_REP,TX_ABD,TNR,TDI,IRD,IED_NIVEL1,IED_NIVEL2,IED_NIVEL3,IED_NIVEL4,...,QT_TUR_INF,QT_TUR_PROF,QT_TUR_PROF_TEC,QT_TUR_EJA,QT_TUR_EJA_FUND,QT_TUR_EJA_MED,QT_TUR_ESP,QT_TUR_ESP_CC,QT_TUR_ESP_CE,VL_OBSERVADO_2021
0,100.0,0.0,0.0,2.0,4.0,2.8,33.4,33.3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,0.0,0
1,100.0,0.0,0.0,9.5,0.0,2.7,75.0,0.0,0.0,25.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
2,67.5,32.5,0.0,7.0,4.5,3.2,50.0,0.0,0.0,50.0,...,1.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0
3,96.9,3.1,0.0,1.5,4.5,3.4,50.0,0.0,0.0,50.0,...,1.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0
4,78.9,21.1,0.0,17.4,60.0,3.0,50.0,0.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54271,94.7,5.3,0.0,4.5,32.4,2.4,0.0,2.2,76.6,8.5,...,0.0,0.0,0.0,16.0,10.0,6.0,33.0,33.0,0.0,0
54272,93.9,4.1,2.0,2.8,37.0,2.5,0.0,0.0,80.5,14.6,...,0.0,0.0,0.0,10.0,0.0,10.0,35.0,35.0,0.0,0
54273,98.4,1.6,0.0,4.8,17.5,1.7,0.0,0.0,85.7,14.3,...,0.0,0.0,0.0,0.0,0.0,0.0,13.0,13.0,0.0,0
54274,96.2,3.8,0.0,3.0,23.9,2.4,0.0,19.2,71.2,7.7,...,0.0,0.0,0.0,0.0,0.0,0.0,23.0,23.0,0.0,0


In [7]:


df.drop(['IN_AGUA_FILTRADA','IN_ENERGIA_GERADOR','IN_ENERGIA_OUTROS','IN_LIXO_JOGA_OUTRA_AREA','IN_LIXO_OUTROS','IN_LIXO_RECICLA','IN_BANHEIRO_FORA_PREDIO',
        'IN_BANHEIRO_DENTRO_PREDIO','IN_BERCARIO','IN_DEPENDENCIAS_PNE','IN_LAVANDERIA','QT_SALAS_EXISTENTES','IN_EQUIP_VIDEOCASSETE','IN_EQUIP_RETROPROJETOR','IN_EQUIP_FAX', 
        'IN_EQUIP_FOTO','QT_EQUIP_VIDEOCASSETE','QT_EQUIP_PARABOLICA','QT_EQUIP_COPIADORA','QT_EQUIP_RETROPROJETOR','QT_EQUIP_IMPRESSORA','QT_EQUIP_IMPRESSORA_MULT','QT_EQUIP_FAX',
        'QT_EQUIP_FOTO', 'QT_COMP_ALUNO','QT_COMPUTADOR','QT_COMP_ADMINISTRATIVO','QT_FUNCIONARIOS','IN_MATERIAL_ESP_QUILOMBOLA','IN_MATERIAL_ESP_INDIGENA','IN_MATERIAL_ESP_NAO_UTILIZA',
        'IN_BRASIL_ALFABETIZADO','IN_FINAL_SEMANA','IN_MEDIACAO_PRESENCIAL','IN_REGULAR','IN_FUND','IN_ESP_CE','IN_FUND'],axis=1, inplace = True)

df.reset_index(drop=True, inplace=True)

df

,TX_APR,TX_REP,TX_ABD,TNR,TDI,IRD,IED_NIVEL1,IED_NIVEL2,IED_NIVEL3,IED_NIVEL4,...,QT_TUR_INF,QT_TUR_PROF,QT_TUR_PROF_TEC,QT_TUR_EJA,QT_TUR_EJA_FUND,QT_TUR_EJA_MED,QT_TUR_ESP,QT_TUR_ESP_CC,QT_TUR_ESP_CE,VL_OBSERVADO_2021
0,100.0,0.0,0.0,2.0,4.0,2.8,33.4,33.3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,0.0,0
1,100.0,0.0,0.0,9.5,0.0,2.7,75.0,0.0,0.0,25.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
2,67.5,32.5,0.0,7.0,4.5,3.2,50.0,0.0,0.0,50.0,...,1.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0
3,96.9,3.1,0.0,1.5,4.5,3.4,50.0,0.0,0.0,50.0,...,1.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0
4,78.9,21.1,0.0,17.4,60.0,3.0,50.0,0.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54271,94.7,5.3,0.0,4.5,32.4,2.4,0.0,2.2,76.6,8.5,...,0.0,0.0,0.0,16.0,10.0,6.0,33.0,33.0,0.0,0
54272,93.9,4.1,2.0,2.8,37.0,2.5,0.0,0.0,80.5,14.6,...,0.0,0.0,0.0,10.0,0.0,10.0,35.0,35.0,0.0,0
54273,98.4,1.6,0.0,4.8,17.5,1.7,0.0,0.0,85.7,14.3,...,0.0,0.0,0.0,0.0,0.0,0.0,13.0,13.0,0.0,0
54274,96.2,3.8,0.0,3.0,23.9,2.4,0.0,19.2,71.2,7.7,...,0.0,0.0,0.0,0.0,0.0,0.0,23.0,23.0,0.0,0


In [8]:
print(df['VL_OBSERVADO_2021'].value_counts())

0    38169
1    16107
Name: VL_OBSERVADO_2021, dtype: int64


In [9]:
#Balancear as classes, excluindo um percentual de escolas que não atingiram o IDEB
df = df.drop(df[df['VL_OBSERVADO_2021'] == 0].sample(frac=.5780).index)
df.reset_index(inplace=True, drop=True)
print(df['VL_OBSERVADO_2021'].value_counts())

0    16107
1    16107
Name: VL_OBSERVADO_2021, dtype: int64


In [10]:
df.to_csv('datasets/DADOS_ED_PROCESSADOS.csv', sep=';', index=False, encoding='latin-1')

In [11]:
#Separando Dataframes de 20,30 e 40 colunas



df_ed = df.iloc[:,:-1]
alvo = df.iloc[:,-1].values

print(alvo)

#NORMALIZAR O CONJUNTO SEPARADO

scaler = MinMaxScaler()
col_names = df_ed.columns

# Transformar os atributos
df_ed = pd.DataFrame(scaler.fit_transform(df_ed))
df_ed.columns = col_names

df_ed
#Usar o MinMaxScaler no df (com as 298 colunas).
#EXCLUIR A ÚLTIMA COLUNA DO df (ALVO)
#df = mms.fit_transform(df)



#print(alvo.shape)

#print(df20.shape)
#print(df30.shape)
#print(df40.shape)

#df20
#df20.shape


[0 0 0 ... 0 0 0]


,TX_APR,TX_REP,TX_ABD,TNR,TDI,IRD,IED_NIVEL1,IED_NIVEL2,IED_NIVEL3,IED_NIVEL4,...,QT_DOC_ESP_CE,QT_TUR_INF,QT_TUR_PROF,QT_TUR_PROF_TEC,QT_TUR_EJA,QT_TUR_EJA_FUND,QT_TUR_EJA_MED,QT_TUR_ESP,QT_TUR_ESP_CC,QT_TUR_ESP_CE
0,1.000000,0.000000,0.000000,0.100423,0.000000,0.425,0.75,0.000,0.000,0.250,...,0.0,0.025641,0.0,0.0,0.000000,0.000000,0.0,0.015385,0.015385,0.0
1,0.556617,0.490196,0.000000,0.073996,0.049669,0.550,0.50,0.000,0.000,0.500,...,0.0,0.025641,0.0,0.0,0.000000,0.000000,0.0,0.046154,0.046154,0.0
2,0.712142,0.318250,0.000000,0.183932,0.662252,0.500,0.50,0.000,0.000,0.500,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.107692,0.107692,0.0
3,0.944065,0.049774,0.020202,0.019027,0.096026,0.675,0.60,0.300,0.000,0.100,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.076923,0.076923,0.0
4,0.538881,0.500754,0.015152,0.051797,0.185430,0.800,0.30,0.400,0.300,0.000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.215385,0.215385,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32209,0.840382,0.144796,0.053030,0.065539,0.309051,0.375,0.00,0.273,0.636,0.091,...,0.2,0.000000,0.0,0.0,0.091837,0.101124,0.0,0.246154,0.215385,0.1
32210,0.959072,0.045249,0.000000,0.063425,0.395143,0.250,0.00,0.342,0.429,0.229,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.138462,0.138462,0.0
32211,0.854025,0.161388,0.000000,0.057082,0.328918,0.450,0.00,0.390,0.537,0.049,...,0.0,0.000000,0.0,0.0,0.122449,0.134831,0.0,0.323077,0.323077,0.0
32212,0.948158,0.057315,0.000000,0.031712,0.263797,0.350,0.00,0.192,0.712,0.077,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.353846,0.353846,0.0


<h1>Tecnica de agregação de atributos (PCA, principal component analysis) </h1>

In [11]:
#20 colunas
PCA_df = df_ed

pca = PCA(n_components = 150)

pca.fit(PCA_df)
pca.explained_variance_ratio_

array([0.10970697, 0.05102219, 0.03479441, 0.02966243, 0.02556022,
       0.02176565, 0.02030082, 0.01959711, 0.01807896, 0.01709283,
       0.01554777, 0.01456249, 0.01424026, 0.01357159, 0.01324766,
       0.01316557, 0.01273762, 0.01221105, 0.01173701, 0.01156612,
       0.01132817, 0.01114763, 0.01072029, 0.01049999, 0.01024045,
       0.01000101, 0.00986611, 0.00970177, 0.00943417, 0.00939656,
       0.00930508, 0.00915596, 0.00889044, 0.00876538, 0.0086647 ,
       0.00854992, 0.00849641, 0.0082998 , 0.00819935, 0.00807632,
       0.00792771, 0.00780428, 0.00762342, 0.00744588, 0.00730101,
       0.00728739, 0.00700942, 0.00691906, 0.00682454, 0.0067213 ,
       0.00664779, 0.00648264, 0.0064066 , 0.00609627, 0.00602443,
       0.00590651, 0.00585147, 0.0057187 , 0.00565306, 0.00561168,
       0.00554388, 0.00544094, 0.0053043 , 0.00514214, 0.00506116,
       0.00502349, 0.00491328, 0.00483835, 0.00464727, 0.00456814,
       0.00435289, 0.00432503, 0.0041527 , 0.00408351, 0.00394

<h1>Seleção de atributos por ordenação, Utilizando Analise de variacia ANOVA -> f_classif</h1>

In [12]:
f_df = df_ed

fs = SelectKBest(score_func = f_classif, k=10)

fit = fs.fit(f_df,alvo)

features = fit.transform(f_df)

pontuacoes_atributos = fit.scores_

# Obtenha os nomes das colunas do DataFrame original
nomes_colunas = f_df.columns

# Obtenha um vetor booleano indicando quais atributos foram selecionados
atributos_selecionados = fs.get_support()

# Combine os nomes das colunas com as pontuações de cada atributo
pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})

# Filtre apenas os atributos selecionados
atributos_selecionados_pontuacoes = pontuacoes_colunas[atributos_selecionados]

# Imprima as pontuações de cada coluna junto com o nome da coluna correspondente
np.set_printoptions(precision=3,suppress=True)
print(atributos_selecionados_pontuacoes.sort_values(by='Pontuação'))

#score.append("heloo")
# Reduz X para os atributos selecionados

# Resultados


print('\nNúmero original de atributos:', f_df.shape[1])
print('\nNúmero reduzido de atributos:', features.shape[1])

# Exibe as pontuações de cada atributos e os 4 escolhidas (com as pontuações mais altas): preg, plas, mass e age.

print("\nScores dos Atributos Originais: \n", np.sort(fit.scores_)[::-1])

                        Coluna    Pontuação
133           QT_DESKTOP_ALUNO  1461.677472
1                       TX_REP  1569.350345
50      IN_ESGOTO_REDE_PUBLICA  1624.859363
16                    AFD_GRP1  1719.182123
125               QT_EQUIP_SOM  1803.405496
141   IN_INTERNET_APRENDIZAGEM  1969.166779
2                       TX_ABD  2037.800016
212  IN_ORGAO_ASS_PAIS_MESTRES  2614.098436
0                       TX_APR  2956.170328
4                          TDI  7184.002886

Número original de atributos: 253

Número reduzido de atributos: 10

Scores dos Atributos Originais: 
 [     nan      nan 7184.003 2956.17  2614.098 2037.8   1969.167 1803.405
 1719.182 1624.859 1569.35  1461.677 1460.368 1430.046 1422.217 1384.785
 1350.425 1317.045 1239.261 1227.696 1217.816 1180.809 1098.713 1090.286
 1082.985 1069.573 1048.176 1011.176  986.887  986.153  949.293  928.254
  904.209  819.873  810.07   739.281  734.341  729.399  715.229  675.553
  667.065  644.165  630.62   620.869  588.519  

/home/allexandre/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [200 222] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/allexandre/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [31]:
listaMse = [0.53857,0.59714,0.53714,0.58]
algoritmos = ['analise de variancia','qui-quadrado','Mutual','Ordenação recursiva']
MediaAlgoritmos = pd.DataFrame({'Algoritmo': algoritmos, 'Media': listaMse})

MediaAlgoritmos

,Algoritmo,Media
0,analise de variancia,0.53857
1,qui-quadrado,0.59714
2,Mutual,0.53714
3,Ordenação recursiva,0.58000


In [13]:
f_df = df_ed

fs = SelectKBest(score_func = f_classif, k=20)

fit = fs.fit(f_df,alvo)

features = fit.transform(f_df)

pontuacoes_atributos = fit.scores_

# Obtenha os nomes das colunas do DataFrame original
nomes_colunas = f_df.columns

# Obtenha um vetor booleano indicando quais atributos foram selecionados
atributos_selecionados = fs.get_support()

# Combine os nomes das colunas com as pontuações de cada atributo
pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})

# Filtre apenas os atributos selecionados
atributos_selecionados_pontuacoes = pontuacoes_colunas[atributos_selecionados]

# Imprima as pontuações de cada coluna junto com o nome da coluna correspondente
np.set_printoptions(precision=3,suppress=True)
print(atributos_selecionados_pontuacoes.sort_values(by='Pontuação'))
MelhoresColunas20 = atributos_selecionados_pontuacoes.sort_values(by='Pontuação')
MelhoresColunas20.to_csv('datasets/20MelhoresColunas.csv', sep=';', index=False, encoding='latin-1')

#score.append("heloo")
# Reduz X para os atributos selecionados

# Resultados
print('\nNúmero original de atributos:', f_df.shape[1])
print('\nNúmero reduzido de atributos:', features.shape[1])

# Exibe as pontuações de cada atributos e os 4 escolhidas (com as pontuações mais altas): preg, plas, mass e age.

print("\nScores dos Atributos Originais: \n", np.sort(fit.scores_)[::-1])

                         Coluna    Pontuação
56               IN_LIXO_QUEIMA  1180.809072
20                     AFD_GRP5  1217.816085
191  IN_MATERIAL_PED_DESPORTIVA  1227.696386
14                          DSU  1239.261096
82           IN_PARQUE_INFANTIL  1317.045080
139          IN_INTERNET_ALUNOS  1350.424993
53              IN_ESGOTO_FOSSA  1384.785179
85   IN_QUADRA_ESPORTES_COBERTA  1422.217264
22               TP_LOCALIZACAO  1430.045651
84           IN_QUADRA_ESPORTES  1460.368247
133            QT_DESKTOP_ALUNO  1461.677472
1                        TX_REP  1569.350345
50       IN_ESGOTO_REDE_PUBLICA  1624.859363
16                     AFD_GRP1  1719.182123
125                QT_EQUIP_SOM  1803.405496
141    IN_INTERNET_APRENDIZAGEM  1969.166779
2                        TX_ABD  2037.800016
212   IN_ORGAO_ASS_PAIS_MESTRES  2614.098436
0                        TX_APR  2956.170328
4                           TDI  7184.002886

Número original de atributos: 253

Número reduzido de 

/home/allexandre/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [200 222] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/allexandre/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [14]:
f_df = df_ed

fs = SelectKBest(score_func = f_classif, k=30)

fit = fs.fit(f_df,alvo)

features = fit.transform(f_df)

pontuacoes_atributos = fit.scores_

# Obtenha os nomes das colunas do DataFrame original
nomes_colunas = f_df.columns

# Obtenha um vetor booleano indicando quais atributos foram selecionados
atributos_selecionados = fs.get_support()

# Combine os nomes das colunas com as pontuações de cada atributo
pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})

# Filtre apenas os atributos selecionados
atributos_selecionados_pontuacoes = pontuacoes_colunas[atributos_selecionados]

# Imprima as pontuações de cada coluna junto com o nome da coluna correspondente
np.set_printoptions(precision=3,suppress=True)
print(atributos_selecionados_pontuacoes.sort_values(by='Pontuação'))

#score.append("heloo")
# Reduz X para os atributos selecionados

# Resultados
print('\nNúmero original de atributos:', f_df.shape[1])
print('\nNúmero reduzido de atributos:', features.shape[1])

# Exibe as pontuações de cada atributos e os 4 escolhidas (com as pontuações mais altas): preg, plas, mass e age.

print("\nScores dos Atributos Originais: \n", np.sort(fit.scores_)[::-1])

                             Coluna    Pontuação
73       IN_BIBLIOTECA_SALA_LEITURA   928.254313
228                     IN_EJA_FUND   949.293208
190      IN_MATERIAL_PED_ARTISTICAS   986.152547
187         IN_MATERIAL_PED_DIFUSAO   986.886590
227                          IN_EJA  1011.175997
63   IN_TRATAMENTO_LIXO_INEXISTENTE  1048.176096
79       IN_LABORATORIO_INFORMATICA  1069.572901
94                IN_SALA_PROFESSOR  1082.985403
58    IN_LIXO_DESTINO_FINAL_PUBLICO  1090.286346
41             IN_AGUA_REDE_PUBLICA  1098.713398
56                   IN_LIXO_QUEIMA  1180.809072
20                         AFD_GRP5  1217.816085
191      IN_MATERIAL_PED_DESPORTIVA  1227.696386
14                              DSU  1239.261096
82               IN_PARQUE_INFANTIL  1317.045080
139              IN_INTERNET_ALUNOS  1350.424993
53                  IN_ESGOTO_FOSSA  1384.785179
85       IN_QUADRA_ESPORTES_COBERTA  1422.217264
22                   TP_LOCALIZACAO  1430.045651
84               IN_

/home/allexandre/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [200 222] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/allexandre/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


<h1>Seleção de atributos por ordenação, Utilizando metodo de seleção Qui-quadrado (Chi-Squared) chi 2</h1>

In [15]:
# Escalando os dados agora com o método MinMax - Apenas para fins de compreensão
f_df = df_ed


# Método de seleção: Qui-quadrado (Chi-Squared) -> chi2
# k = números de atributos que se deseja selecionar no ranking
fs = SelectKBest(score_func = chi2, k = 10)
fit = fs.fit(f_df, alvo)

pontuacoes_atributos = fit.scores_

# Obtenha os nomes das colunas do DataFrame original
nomes_colunas = f_df.columns

# Obtenha um vetor booleano indicando quais atributos foram selecionados
atributos_selecionados = fs.get_support()

# Combine os nomes das colunas com as pontuações de cada atributo
pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})
# Filtre apenas os atributos selecionados
atributos_selecionados_pontuacoes = pontuacoes_colunas[atributos_selecionados]
# Imprima as pontuações de cada coluna junto com o nome da coluna correspondente
np.set_printoptions(precision=3,suppress=True)
print(atributos_selecionados_pontuacoes.sort_values(by='Pontuação'))

# Aplicando a seleção

                            Coluna    Pontuação
85      IN_QUADRA_ESPORTES_COBERTA   675.119645
4                              TDI   719.626250
228                    IN_EJA_FUND   741.570613
53                 IN_ESGOTO_FOSSA   759.478967
227                         IN_EJA   760.866112
58   IN_LIXO_DESTINO_FINAL_PUBLICO   869.189938
82              IN_PARQUE_INFANTIL   977.068665
56                  IN_LIXO_QUEIMA  1040.184417
22                  TP_LOCALIZACAO  1050.194992
212      IN_ORGAO_ASS_PAIS_MESTRES  1271.466383


In [16]:
# Escalando os dados agora com o método MinMax - Apenas para fins de compreensão
f_df = df_ed


# Método de seleção: Qui-quadrado (Chi-Squared) -> chi2
# k = números de atributos que se deseja selecionar no ranking
fs = SelectKBest(score_func = chi2, k = 20)
fit = fs.fit(f_df, alvo)

pontuacoes_atributos = fit.scores_

# Obtenha os nomes das colunas do DataFrame original
nomes_colunas = f_df.columns

# Obtenha um vetor booleano indicando quais atributos foram selecionados
atributos_selecionados = fs.get_support()

# Combine os nomes das colunas com as pontuações de cada atributo
pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})
# Filtre apenas os atributos selecionados
atributos_selecionados_pontuacoes = pontuacoes_colunas[atributos_selecionados]
MelhoresColunas20 = atributos_selecionados_pontuacoes.sort_values(by='Pontuação')
MelhoresColunas20.to_csv('datasets/20MelhoresColunasChi2.csv', sep=';', index=False, encoding='latin-1')

# Imprima as pontuações de cada coluna junto com o nome da coluna correspondente
np.set_printoptions(precision=3,suppress=True)
print(atributos_selecionados_pontuacoes.sort_values(by='Pontuação'))

                            Coluna    Pontuação
223                     IN_NOTURNO   421.071642
214     IN_ORGAO_GREMIO_ESTUDANTIL   430.249407
184     IN_MATERIAL_PED_MULTIMIDIA   434.510742
79      IN_LABORATORIO_INFORMATICA   435.618254
42          IN_AGUA_POCO_ARTESIANO   494.003618
84              IN_QUADRA_ESPORTES   500.705840
190     IN_MATERIAL_PED_ARTISTICAS   554.266396
141       IN_INTERNET_APRENDIZAGEM   568.377304
139             IN_INTERNET_ALUNOS   585.919782
50          IN_ESGOTO_REDE_PUBLICA   642.945073
85      IN_QUADRA_ESPORTES_COBERTA   675.119645
4                              TDI   719.626250
228                    IN_EJA_FUND   741.570613
53                 IN_ESGOTO_FOSSA   759.478967
227                         IN_EJA   760.866112
58   IN_LIXO_DESTINO_FINAL_PUBLICO   869.189938
82              IN_PARQUE_INFANTIL   977.068665
56                  IN_LIXO_QUEIMA  1040.184417
22                  TP_LOCALIZACAO  1050.194992
212      IN_ORGAO_ASS_PAIS_MESTRES  1271

In [17]:
# Escalando os dados agora com o método MinMax - Apenas para fins de compreensão
f_df = df_ed


# Método de seleção: Qui-quadrado (Chi-Squared) -> chi2
# k = números de atributos que se deseja selecionar no ranking
fs = SelectKBest(score_func = chi2, k = 30)
fit = fs.fit(f_df, alvo)

pontuacoes_atributos = fit.scores_

# Obtenha os nomes das colunas do DataFrame original
nomes_colunas = f_df.columns

# Obtenha um vetor booleano indicando quais atributos foram selecionados
atributos_selecionados = fs.get_support()

# Combine os nomes das colunas com as pontuações de cada atributo
pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})
# Filtre apenas os atributos selecionados
atributos_selecionados_pontuacoes = pontuacoes_colunas[atributos_selecionados]

# Imprima as pontuações de cada coluna junto com o nome da coluna correspondente
np.set_printoptions(precision=3,suppress=True)
print(atributos_selecionados_pontuacoes.sort_values(by='Pontuação'))

                            Coluna    Pontuação
87                   IN_REFEITORIO   304.711893
186     IN_MATERIAL_PED_CIENTIFICO   324.075773
142         IN_INTERNET_COMUNIDADE   349.163934
187        IN_MATERIAL_PED_DIFUSAO   357.193584
128         IN_EQUIP_LOUSA_DIGITAL   360.444403
101     IN_ACESSIBILIDADE_CORRIMAO   363.385603
51         IN_ESGOTO_FOSSA_SEPTICA   366.494055
193         IN_MATERIAL_PED_ETNICO   404.081484
52           IN_ESGOTO_FOSSA_COMUM   404.616661
20                        AFD_GRP5   407.743971
223                     IN_NOTURNO   421.071642
214     IN_ORGAO_GREMIO_ESTUDANTIL   430.249407
184     IN_MATERIAL_PED_MULTIMIDIA   434.510742
79      IN_LABORATORIO_INFORMATICA   435.618254
42          IN_AGUA_POCO_ARTESIANO   494.003618
84              IN_QUADRA_ESPORTES   500.705840
190     IN_MATERIAL_PED_ARTISTICAS   554.266396
141       IN_INTERNET_APRENDIZAGEM   568.377304
139             IN_INTERNET_ALUNOS   585.919782
50          IN_ESGOTO_REDE_PUBLICA   642

In [18]:
# Escalando os dados agora com o método MinMax - Apenas para fins de compreensão
f_df = df_ed


# Método de seleção: Qui-quadrado (Chi-Squared) -> chi2
# k = números de atributos que se deseja selecionar no ranking
fs = SelectKBest(score_func = mutual_info_classif, k = 10)
fit = fs.fit(f_df, alvo)

pontuacoes_atributos = fit.scores_

# Obtenha os nomes das colunas do DataFrame original
nomes_colunas = f_df.columns

# Obtenha um vetor booleano indicando quais atributos foram selecionados
atributos_selecionados = fs.get_support()

# Combine os nomes das colunas com as pontuações de cada atributo

pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})

# Filtre apenas os atributos selecionados
atributos_selecionados_pontuacoes = pontuacoes_colunas[atributos_selecionados]

# Imprima as pontuações de cada coluna junto com o nome da coluna correspondente
np.set_printoptions(precision=3,suppress=True)
print(atributos_selecionados_pontuacoes.sort_values(by='Pontuação'))



                             Coluna  Pontuação
16                         AFD_GRP1   0.028455
133                QT_DESKTOP_ALUNO   0.029190
63   IN_TRATAMENTO_LIXO_INEXISTENTE   0.029856
141        IN_INTERNET_APRENDIZAGEM   0.032952
1                            TX_REP   0.033299
125                    QT_EQUIP_SOM   0.034255
212       IN_ORGAO_ASS_PAIS_MESTRES   0.038069
2                            TX_ABD   0.048017
0                            TX_APR   0.062202
4                               TDI   0.118936


In [19]:
# Escalando os dados agora com o método MinMax - Apenas para fins de compreensão
f_df = df_ed


# Método de seleção: Qui-quadrado (Chi-Squared) -> chi2
# k = números de atributos que se deseja selecionar no ranking
fs = SelectKBest(score_func = mutual_info_classif, k = 20)
fit = fs.fit(f_df, alvo)

pontuacoes_atributos = fit.scores_

# Obtenha os nomes das colunas do DataFrame original
nomes_colunas = f_df.columns

# Obtenha um vetor booleano indicando quais atributos foram selecionados
atributos_selecionados = fs.get_support()

# Combine os nomes das colunas com as pontuações de cada atributo

pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})

# Filtre apenas os atributos selecionados
atributos_selecionados_pontuacoes = pontuacoes_colunas[atributos_selecionados]
MelhoresColunas20 = atributos_selecionados_pontuacoes.sort_values(by='Pontuação')
MelhoresColunas20.to_csv('datasets/20MelhoresColunasMutual.csv', sep=';', index=False, encoding='latin-1')

# Imprima as pontuações de cada coluna junto com o nome da coluna correspondente
np.set_printoptions(precision=3,suppress=True)
print(atributos_selecionados_pontuacoes.sort_values(by='Pontuação'))

                             Coluna  Pontuação
139              IN_INTERNET_ALUNOS   0.022897
82               IN_PARQUE_INFANTIL   0.022973
79       IN_LABORATORIO_INFORMATICA   0.023448
22                   TP_LOCALIZACAO   0.024640
60     IN_TRATAMENTO_LIXO_SEPARACAO   0.025059
63   IN_TRATAMENTO_LIXO_INEXISTENTE   0.025692
56                   IN_LIXO_QUEIMA   0.025836
133                QT_DESKTOP_ALUNO   0.025838
85       IN_QUADRA_ESPORTES_COBERTA   0.026351
84               IN_QUADRA_ESPORTES   0.026764
50           IN_ESGOTO_REDE_PUBLICA   0.027275
143   IN_ACESSO_INTERNET_COMPUTADOR   0.029059
125                    QT_EQUIP_SOM   0.030144
141        IN_INTERNET_APRENDIZAGEM   0.031872
16                         AFD_GRP1   0.032194
1                            TX_REP   0.034950
212       IN_ORGAO_ASS_PAIS_MESTRES   0.040321
2                            TX_ABD   0.050963
0                            TX_APR   0.065072
4                               TDI   0.120727


In [20]:
# Escalando os dados agora com o método MinMax - Apenas para fins de compreensão
f_df = df_ed


# Método de seleção: Qui-quadrado (Chi-Squared) -> chi2
# k = números de atributos que se deseja selecionar no ranking
fs = SelectKBest(score_func = mutual_info_classif, k = 30)
fit = fs.fit(f_df, alvo)

pontuacoes_atributos = fit.scores_

# Obtenha os nomes das colunas do DataFrame original
nomes_colunas = f_df.columns

# Obtenha um vetor booleano indicando quais atributos foram selecionados
atributos_selecionados = fs.get_support()

# Combine os nomes das colunas com as pontuações de cada atributo

pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})

# Filtre apenas os atributos selecionados
atributos_selecionados_pontuacoes = pontuacoes_colunas[atributos_selecionados]

# Imprima as pontuações de cada coluna junto com o nome da coluna correspondente
np.set_printoptions(precision=3,suppress=True)
print(atributos_selecionados_pontuacoes.sort_values(by='Pontuação'))

                             Coluna  Pontuação
184      IN_MATERIAL_PED_MULTIMIDIA   0.017945
140      IN_INTERNET_ADMINISTRATIVO   0.018189
132                IN_DESKTOP_ALUNO   0.018332
238                      QT_DOC_EJA   0.018784
22                   TP_LOCALIZACAO   0.019974
53                  IN_ESGOTO_FOSSA   0.020364
94                IN_SALA_PROFESSOR   0.020745
187         IN_MATERIAL_PED_DIFUSAO   0.021465
143   IN_ACESSO_INTERNET_COMPUTADOR   0.021496
139              IN_INTERNET_ALUNOS   0.021701
20                         AFD_GRP5   0.022108
41             IN_AGUA_REDE_PUBLICA   0.022120
145                   TP_REDE_LOCAL   0.022266
85       IN_QUADRA_ESPORTES_COBERTA   0.023109
56                   IN_LIXO_QUEIMA   0.023337
63   IN_TRATAMENTO_LIXO_INEXISTENTE   0.023870
127                     QT_EQUIP_TV   0.024208
191      IN_MATERIAL_PED_DESPORTIVA   0.024618
14                              DSU   0.026025
84               IN_QUADRA_ESPORTES   0.026604
141        IN

In [21]:
f_df = df_ed

rf = RandomForestClassifier()
    
n_features = 10

fs = RFE(rf, n_features_to_select = n_features)

fit = fs.fit(f_df,alvo)

In [22]:
pontuacoes_atributos = fit.ranking_

nomes_colunas = f_df.columns

atributos_selecionados = fs.support_

pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})

atributos_selecionados_pontuacoes = pontuacoes_colunas[atributos_selecionados]

print(atributos_selecionados_pontuacoes.sort_values(by='Pontuação'))

np.set_printoptions(precision=3,suppress=True)
print("Num Atributos: %s" % (fit.n_features_))
print("\nNomes dos Atributos Selecionados: %s" % fit.get_feature_names_out(input_features=f_df.columns))
#print("Atributos selecionados: %s" % (fit.support_))
#print("Ranking Atributos: %s" % (fit.ranking_))

               Coluna  Pontuação
0              TX_APR          1
4                 TDI          1
8          IED_NIVEL3          1
9          IED_NIVEL4          1
15                ATU          1
16           AFD_GRP1          1
18           AFD_GRP3          1
133  QT_DESKTOP_ALUNO          1
232       QT_MAT_FUND          1
242     QT_DOC_ESP_CC          1
Num Atributos: 10

Nomes dos Atributos Selecionados: ['TX_APR' 'TDI' 'IED_NIVEL3' 'IED_NIVEL4' 'ATU' 'AFD_GRP1' 'AFD_GRP3'
 'QT_DESKTOP_ALUNO' 'QT_MAT_FUND' 'QT_DOC_ESP_CC']


In [21]:
f_df = df_ed


np.set_printoptions(precision=3,suppress=True)
rf = RandomForestClassifier()

rf.fit(f_df,alvo)

importances = rf.feature_importances_
feature_names = f_df.columns
feature_importance_df = pd.DataFrame({'Feature':feature_names,'Importance':importances})

feature_importance_df = feature_importance_df.sort_values(by='Importance',ascending = False)

pd.set_option('display.max_rows',None)
print(feature_importance_df)



                            Feature  Importance
4                               TDI    0.083507
0                            TX_APR    0.037823
2                            TX_ABD    0.025111
16                         AFD_GRP1    0.019177
212       IN_ORGAO_ASS_PAIS_MESTRES    0.019165
1                            TX_REP    0.018214
3                               TNR    0.017221
125                    QT_EQUIP_SOM    0.017001
15                              ATU    0.016554
232                     QT_MAT_FUND    0.015961
133                QT_DESKTOP_ALUNO    0.015872
9                        IED_NIVEL4    0.014925
18                         AFD_GRP3    0.014026
8                        IED_NIVEL3    0.013614
5                               IRD    0.013491
235                     QT_DOC_FUND    0.013392
242                   QT_DOC_ESP_CC    0.012635
241                      QT_DOC_ESP    0.012250
127                     QT_EQUIP_TV    0.011968
7                        IED_NIVEL2    0

In [29]:
# Extract feature importances
importances = fit.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

# Rank features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

# Select top N features (example selecting top 10 features)
top_features = feature_importance_df['Feature'][:10].values
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]


MelhoresColunas20 = atributos_selecionados_pontuacoes.sort_values(by='Pontuação')
MelhoresColunas20.to_csv('datasets/20MelhoresColunasRf.csv', sep=';', index=False, encoding='latin-1')

print("Num Atributos: %s" % (fit.n_features_))
print("\nNomes dos Atributos Selecionados: %s" % fit.get_feature_names_out(input_features=f_df.columns))
#print("Atributos selecionados: %s" % (fit.support_))
#print("Ranking Atributos: %s" % (fit.ranking_))

AttributeError: 'RFE' object has no attribute 'feature_importances_'

In [ ]:
f_df = df_ed

rf = RandomForestClassifier()
    
n_features = 30

fs = RFE(rf, n_features_to_select = n_features)

fit = fs.fit(f_df,alvo)

In [ ]:
pontuacoes_atributos = fit.ranking_

nomes_colunas = f_df.columns

atributos_selecionados = fs.support_

pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})

atributos_selecionados_pontuacoes = pontuacoes_colunas[atributos_selecionados]

print(atributos_selecionados_pontuacoes.sort_values(by='Pontuação'))

print("Num Atributos: %s" % (fit.n_features_))
print("\nNomes dos Atributos Selecionados: %s" % fit.get_feature_names_out(input_features=f_df.columns))
#print("Atributos selecionados: %s" % (fit.support_))
#print("Ranking Atributos: %s" % (fit.ranking_))

In [ ]:
f_df = df_ed

modelo = LogisticRegression(max_iter=3000);
rfe = RFE(modelo,n_features_to_select = 10)
fit = rfe.fit(f_df,alvo)
print("Atributos selecionados: %s" % fit.support_)

pontuacoes_atributos = fit.ranking_

nomes_colunas = f_df.columns

atributos_selecionados = rfe.support_

pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})

atributos_selecionados_pontuacoes = pontuacoes_colunas[atributos_selecionados]

print(atributos_selecionados_pontuacoes.sort_values(by='Pontuação'))



In [ ]:
f_df = df_ed
modelo = ExtraTreesClassifier(n_estimators=100)
fit = modelo.fit(f_df,alvo)

print(modelo.feature_importances_)

pontuacoes_atributos = modelo.feature_importances_

nomes_colunas = f_df.columns


pontuacoes_colunas = pd.DataFrame({'Coluna': nomes_colunas, 'Pontuação': pontuacoes_atributos})

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
print(pontuacoes_colunas.sort_values(by='Pontuação')[223:253])

print("\nNomes dos Atributos Selecionados:",pontuacoes_colunas.sort_values(by='Pontuação').columns)
